In [1]:
import pandas as pd
import urllib2
import bs4 as bs
from bs4 import SoupStrainer
import numpy as np
df = pd.read_excel('health.xlsx')

In [2]:
from bs4 import BeautifulSoup
sicf=df['SIC_final'].values
cik=[]
link=[]
comp=[]
sic2=[]

In [3]:
sicf.tolist()

[8011L,
 8021L,
 8031L,
 8041L,
 8042L,
 8043L,
 8049L,
 8051L,
 8052L,
 8059L,
 8062L,
 8063L,
 8069L,
 8071L,
 8072L,
 8082L,
 8092L,
 8093L,
 8099L]

In [4]:
def webdata(url,s):
    a=[]
    html = urllib2.urlopen(url).read()
    if(html.find("Companies for ")>0):
        soup = BeautifulSoup(html,'lxml')
        for c in soup.find_all('a', href=True, text=True, id=''):
            if "getcompany&CIK" in c['href']:
                link.append('https://www.sec.gov'+c['href'])
        rows = soup.find_all('tr')   #Searching for all <tr>
        for row in rows:                   
            td_cells = row.find_all('td')   #Within tr searching for td
            for td_cell in td_cells:
                a.append(td_cell.text)
        i=0
        while i<len(a):
            cik.append(a[i])
            comp.append(a[i+1])
            sic2.append(s)
            i=i+3
        if(soup.find("input", value=True)):
            if(soup.find("input", value=True)['value']=='Previous 100'):
                if(len(soup.find_all("input", value=True))>1):
                    html2=str(soup.find_all("input", value=True)[1])
                    soup1 = BeautifulSoup(html2,'lxml')
                    x= soup1.find("input", value=True)["onclick"]
                    url_temp='https://www.sec.gov'+ x[17:len(x)-1]
                    webdata(url_temp,s)
            else:
                x= soup.find("input", value=True)["onclick"]
                url_temp='https://www.sec.gov'+ x[17:len(x)-1]
                webdata(url_temp,s)

In [5]:
for k in sicf:
    url='https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&SIC=' + str(k) + '&owner=exclude&match=&start=0&count=100&hidefilings=0'
    webdata(url,k)

In [6]:
outputdf=pd.DataFrame(comp, columns=['Company Name'])
outputdf['CIK']=cik
outputdf['SIC']= sic2
outputdf['weblink']= link
#outputdf.to_csv('healthcik.csv')

In [7]:
company=comp
final_url=[]
date=[]
type_form=[] 
company_name=[]
url_search=[]
def datasort(html,m):
    a=[]
    soup = BeautifulSoup(html)    #Converting html to soup object
    rows = soup.find_all('tr')   #Searching for all <tr>
    for row in rows:                   
        td_cells = row.find_all('td')   #Within tr searching for td
        for td_cell in td_cells:
            a.append(td_cell.text)    #Within td appending all the text in a list
    k=0
    while k<len(a):
        if "Annual report" in a[k] or "Transition reports" in a[k] :   #Searching for files
            type_form.append(a[k-2])  #Appending filing type and date based on the above criteria
            date.append(a[k+1])
        k=k+1
    for c in soup.find_all('a', href=True, text=True, id='documentsbutton'):
        if "/Archives/edgar/data/" in c['href']:   #Looking for links
            final_url.append(c['href'])
            company_name.append(m)
company_temp1=[]
for c in company:
    company_temp=c
    c=c.replace("&","%26")
    c=c.replace(" ","+")
    url="https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company="+c+"&type=10-K&dateb=&owner=exclude&count=10000"
    html = urllib2.urlopen(url).read()
    if(html.find("Companies with names matching")>0):        #Identfying companies for which we dont get the desired page directly
        while html.find('<a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=') !=-1:
            start_temp = html.find('<a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=')
            end_temp = html.find('<acronym title="Standard Industrial Code">SIC</acronym>')
            url_temp=html[start_temp+57:start_temp+67]
            url_search.append('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='+url_temp+'&owner=exclude&count=10000&hidefilings=0&type=10-K')
            company_temp1.append(company_temp)
            html=html[end_temp+200:]
    elif(html.find("Companies with names matching") <1 and html.find("No matching companies") <1):
        datasort(html,company_temp)
n=0
for u in url_search:
    html = urllib2.urlopen(u).read()
    datasort(html,company_temp1[n])
    n=n+1

C:\Users\Lenovo\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\Lenovo\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [8]:
outputdf=pd.DataFrame(company_name, columns=['Company Name'])
outputdf['Date of Filing']=date
outputdf['Filing Type']= type_form
outputdf['weblink']= final_url

outputdf=outputdf[outputdf['Filing Type']=='10-K']
outputdf.to_csv('10K_Links_health.csv')